In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100

在计算卡组强度时，LL Helper引入了“通用强度”的概念，本文通过整理LL Helper的代码来深入研究各种强度的计算方式，并且尝试构建一个统一的计算体系来为代码维护和将来自动选择最强卡组并匹配宝石的功能提供理论基础。

# 1 游戏界面单张强度数值的计算方式
<img src="Card_Detail_UI.png" width="480">


我们以这张千歌为例，这张卡的基础三围是(5340,4240,4320)，绊为1000，并且装备了红十字（单体加成16%）和红香水（单体增加450点）两个宝石。

首先，每张卡有其【基础三围】和【绊】，通过这两项可以计算【含绊三围】，由于绊只影响卡的主属性，具体计算方式如下

$$【含绊三围】(\text{attr}) = 【基础三围】(\text{attr}) + 【绊】\times \mathbb{I} (\text{attr}=主属性) $$

其中$\mathbb{I}(\cdot)$为indicator function，当条件满足是为1，反之为0。
所以这张千歌的含绊三围为 (5340+1000,4240,4320) = (6340,4240,4320)。

在游戏中选取这张卡时会出现上面的画面，可以看到游戏内显示的【单卡界面三围】是(7805,4240,4320)。

$$【单卡界面三围】(\text{attr}) = 【含绊三围】(\text{attr}) + \sum_{\text{gem} \in 同属性比例加成宝石} \mathsf{ceil}(【含绊三围】(\text{attr}) \times \text{gem}的加成比例) + \sum_{\text{gem} \in 同属性数值加成宝石} \text{gem}的加成数值 $$

值得注意的是计算顺序是先向上取整再相加，所以如果同时装备了同色的指环和十字的话可能会比先加成26%再向上取整多一点点。

In [2]:
# 计算【含绊三围】
base_attr = np.array([5340, 4240, 4320])
bond = np.array([1000, 0, 0])
base_bond_attr = base_attr + bond
# 计算【单卡界面三围】
smile_perfume = np.array([450, 0, 0])
smile_cross = np.array([0.16, 0, 0])
card_attr = base_bond_attr + np.ceil(base_bond_attr*smile_cross) + smile_perfume

print('【单卡界面三围】', card_attr.tolist())

【单卡界面三围】 [7805.0, 4240.0, 4320.0]


# 2 游戏界面组队强度数值的计算方式

在计算组队强度时，由于队伍center技能、全体加成宝石，以及好友应援center技能的影响，计算变得复杂来许多。

首先忽视所有技能宝石，找出所有全体加成宝石将其集合【团队宝石组】，然后对每张卡计算其【团队加成点数】以及【团队C前三围】

$$【团队加成点数】(\text{attr}) = \sum_{\text{gem} \in 团队宝石组中的同属性宝石} \mathsf{ceil}(【含绊三围】(\text{attr}) \times \text{gem}的加成比例) $$
$$【团队\text{C}前三围】(\text{attr}) = 【单卡界面三围】(\text{attr}) + 【团队加成点数】(\text{attr}) $$

对于每张卡，计算其在Center技能下的加成点数。
由于有Center技能和好友Center技能应援两个因素，并且SSR和UR的Center技能分为主C和副C两部分，我们将【Center加成点数】分为四部分【团队主C加成点数】，【团队副C加成点数】，【好友主C加成点数】，【好友副C加成点数】计算：

$$ 【团队主\text{C}加成点数】(\text{attr}) = [【团队\text{C}前三围】(\text{attr}) \times 团队主\text{C}加成比例] \\ 
  【团队副\text{C}加成点数】(\text{attr}) = [【团队\text{C}前三围】(\text{attr}) \times 团队副\text{C}加成比例] \\
  【好友主\text{C}加成点数】(\text{attr}) = [【团队\text{C}前三围】(\text{attr}) \times 好友主\text{C}加成比例] \\
  【好友副\text{C}加成点数】(\text{attr}) = [【团队\text{C}前三围】(\text{attr}) \times 好友副\text{C}加成比例] \\
  【\text{Center}加成点数】(\text{attr}) = 【团队主\text{C}加成点数】(\text{attr}) + 【团队副\text{C}加成点数】(\text{attr}) + 【好友主\text{C}加成点数】(\text{attr}) + 好友副\text{C}加成点数】(\text{attr}) $$
  
最后，每张卡的【团队真实三围】为：

$$ 【团队真实三围】(\text{attr}) = 【团队\text{C}前三围】(\text{attr}) + 【\text{Center}加成点数】(\text{attr}) $$

到此为止，我们有了九张卡的【团队加成点数】、【Center加成点数】以及【团队真实三围】，那么在游戏界面下

* 【主唱技能提升值】(Center Skill Bonus)为九张卡【Center加成点数】之和
* 【团队宝石提升值】(Center SIS Bonus)为九张卡【团队加成点数】之和
* 【团队强度】(Team Total)为九张卡【团队真实三围】之和。

我们以下图为例，此例中没有好友应援，所以【好友主C加成点数】和【好友副C加成点数】均为零。
<img src="Team_Detail_UI1.png" width="600">
<img src="Team_Detail_UI2.png" width="600">

In [3]:
# 计算【含绊三围】
base_attr = np.array([[5040, 3790, 4090], [5140, 4290, 3490], [5140, 3380, 4400], 
					  [5310, 3870, 4700], [5110, 3470, 4290], [5330, 4080, 4400],
					  [5070, 4160, 3780], [5140, 3400, 4410], [5340, 4240, 4320]])
bond = np.array([[500, 0, 0], [750, 0 , 0], [750, 0, 0], [1000, 0, 0], [750, 0, 0],
				 [1000, 0, 0], [500, 0, 0], [750, 0, 0], [1000, 0, 0]])
base_bond_attr = base_attr + bond
# 计算【单卡界面三围】
smile_ring = np.zeros((9,3))
smile_ring[[3,5,8],0] = 0.1
smile_cross = np.zeros((9,3))
smile_cross[[1,2,3,5,7,8],0] = 0.16
card_attr = base_bond_attr + np.ceil(base_bond_attr*smile_ring) + np.ceil(base_bond_attr*smile_cross)
# 计算【团队C前三围】
smile_aura_num, smile_veil_num = 3, 2
smile_aura = np.array([0.018,0,0]).reshape((1,-1))
smile_veil = np.array([0.024,0,0]).reshape((1,-1))
team_bonus = np.ceil(base_bond_attr*smile_aura)*smile_aura_num + np.ceil(base_bond_attr*smile_veil)*smile_veil_num
before_center_attr = card_attr + team_bonus
# 计算【Center加成点数】
team_main_C = np.array([0.09,0,0]).reshape((1,-1))
team_vice_C = np.zeros((9,3))
team_vice_C[[0,1,3,4,7,8],0] = 0.06
center_bonus = np.ceil(before_center_attr*team_main_C) + np.ceil(before_center_attr*team_vice_C)
# 【计算团队强度】，【主唱技能提升值】，【团队宝石提升值】
final_attr = before_center_attr + center_bonus
team_total = final_attr.sum(axis=0)
center_skill_bonus = center_bonus.sum(axis=0)
team_gem_bonus = team_bonus.sum(axis=0)

print('【单卡界面三围】：')
print('\n'.join([str(x) for x in card_attr.tolist()]))
print('【团队强度】', team_total.tolist())
print('【主唱技能提升值】', center_skill_bonus.tolist())
print('【团队宝石提升值】', team_gem_bonus.tolist())

【单卡界面三围】：
[5540.0, 3790.0, 4090.0]
[6833.0, 4290.0, 3490.0]
[6833.0, 3380.0, 4400.0]
[7951.0, 3870.0, 4700.0]
[5860.0, 3470.0, 4290.0]
[7976.0, 4080.0, 4400.0]
[5570.0, 4160.0, 3780.0]
[6833.0, 3400.0, 4410.0]
[7989.0, 4240.0, 4320.0]
【团队强度】 [75587.0, 34680.0, 37880.0]
【主唱技能提升值】 [8707.0, 0.0, 0.0]
【团队宝石提升值】 [5495.0, 0.0, 0.0]


# 3 游戏得分的计算方式
由于网上有不同版本的计算参数体系（根本原因是选择【分数/强度系数】为0.01或0.0125），两组参数如果混用会造成错误的结果。

首先我们需要固定一组参数：

$ 【分数/强度系数】: \theta = 0.01 $

$ 【判定系数】: 
  \lambda(\text{Perfect}) = 1.25, \quad
  \lambda(\text{Great}) = 1.10, \quad
  \lambda(\text{Good}) = 1.00, \quad
  \lambda(\text{Bad}) = 0.50, \quad
  \lambda(\text{Miss}) = 0.00 $
  
$ 【音符种类系数】: \tau(单点) = 1, \quad \tau(长音符) = 1, \quad \tau(滑键) = 0.5 $ （注：其实游戏中滑键的最后可能是长音符，但是由于数量非常少这里我们归入滑键）

$ 【连击系数】: \chi(连击数) = 
    \begin{cases} 
        1.00, \quad \text{if}\,\, 1 \le 连击数 \le 50 \\
        1.10, \quad \text{if}\,\, 51 \le 连击数 \le 100 \\
        1.15, \quad \text{if}\,\, 101 \le 连击数 \le 200 \\
        1.20, \quad \text{if}\,\, 201 \le 连击数 \le 400 \\
        1.25, \quad \text{if}\,\, 401 \le 连击数 \le 600 \\
        1.30, \quad \text{if}\,\, 601 \le 连击数 \le 800 \\
        1.35, \quad \text{if}\,\, 连击数 \ge 801 \\
    \end{cases} $

$ 【同团加成系数】: \mu_{团} = 1.1, \qquad 【同色加成系数】: \mu_{色} = 1.1 $

值得注意的是，考虑到诡计宝石的存在，在判定加强状态存在期间，装备有诡计宝石的卡会提升【单卡界面三围】中33%的对应属性（意味着诡计宝石不受Center技能和全体宝石的加成），因此我们引入【判定时团队强度】的概念：

$ 【判定时团队强度】(\text{attr}) = \sum_{i=1}^9 {\Large[} 卡i的【单卡界面三围】(\text{attr}) \times (1 + 0.33 \times \mathbb{I}(卡i装备 \text{attr} 属性的诡计宝石)) \\
\qquad \qquad \qquad \qquad \qquad \qquad + 卡i的【团队加成点数】(\text{attr}) + 卡i的【\text{Center}加成点数】(\text{attr}) {\Large]} $

给定Live歌曲以及组队，首先我们可以确定各个位置上的

$【同色同团加成系数】: \mu(i) = \mu_{\mathrm{团}}^{\mathbb{I}(卡i与歌曲同团)} \quad \,\, \times \,\, \mu_{\mathrm{色}}^{\mathbb{I}(卡i与歌曲同色)} \quad \,\,, \qquad \forall i \in \{1, \ldots, 9\} $

* 在按下普通音符或滑键时，系统会得知该音符的位置 $\text{pos}$、判定 $\text{judge}$、种类 $\text{type}$，当前连击数 $\text{combo}$，以及这时判定加强状态是否存在，因此这个音符的得分可以计算为

  $$ \mathsf{floor}(S \times \theta \times \lambda(\text{judge}) \times \tau(\text{type}) \times \chi(\text{combo}) \times \mu(\text{pos})) $$
  
* 对于长音符，其判定会取首尾判定比较差的那个(这个不会影响得分，但会影响P系技能的发动)，但是它对应的判定系数是首位判定系数的乘积
(比如头是Perfect尾是Great，那么这个长音符的判定为Great，它对应的判定系数为 $1.25 \times 1.10$)。
  在长音符的尾部抬起时，系统会得知该音符的位置 $\text{pos}$、首判定 $\text{judge1}$、尾判定 $\text{judge2}$、种类 $\text{type}$，当前连击数 $\text{combo}$，以及这时判定加强状态是否存在，因此这个音符的得分可以计算为
  
  $$ \mathsf{floor}( S \times \theta \times \lambda(\text{judge1}) \times \lambda(\text{judge2}) \times \tau(\text{type}) \times \chi(\text{combo}) \times \mu(\text{pos}) ) $$
  
其中 $S$ 为对应Live属性的【当前团队强度】
$$ S = 
    \begin{cases} 
        【团队强度】(\text{live attr})，&如果判定加强状态不存在 \\
        【判定时团队强度】(\text{live attr})，&如果判定加强状态存在
    \end{cases} $$

举例：这里使用[LoveLive Wiki](https://www.lovelivewiki.com/w/%E5%88%86%E6%95%B0%E8%AE%A1%E7%AE%97%E6%96%B9%E5%BC%8F)的例子。

在僕らのLIVE 君とのLIFE(Easy)全Perfect判定，队伍成员均为μ's成员奶卡，并且没有装备任何宝石，将【团队强度】的Smile属性数值按照20000进行计算。
谱面数据里总连击数为95，其中第1,52,76,89,93是长音符。

In [4]:
#【分数/强度系数】
theta = 0.01 
#【判定系数】
lam_p, lam_g = 1.25, 1.1
#【音符种类系数】
tau_tap, tau_long, tau_swing = 1, 1, 0.5
#【连击系数】
combo_bonus  = { 0:1.00,	 1:1.10,	 2:1.15,	 3:1.15,
				 4:1.20,	 5:1.20,	 6:1.20,	 7:1.20,
				 8:1.25,	 9:1.25,	10:1.25,	11:1.25,
				12:1.30,	13:1.30,	14:1.30,	15:1.30}
def chi(combo):
	return 1.35 if combo > 800 else combo_bonus[int((combo-1)/50)]
#【同团加成系数】,【同色加成系数】
mu_g, mu_c = 1.1, 1.1

N, S = 95, 20000
is_long = [x in [1,52,76,89,93] for x in range(1,N+1)]
judge_factor = [lam_p**(1+v) for v in is_long]
type_factor = [tau_long**v for v in is_long]
combo_factor = [chi(x) for x in range(1,N+1)]
total_score = 0
for jf,tf,cf in zip(judge_factor, type_factor, combo_factor):
	total_score += np.floor(S * theta * jf * tf * cf * mu_g * mu_c)
print('最终得分为', total_score)

最终得分为 30448.0


# 4 技能收益和技能强度的计算方式
由于技能的触发方式为时间(T系)、音符数(N系)、连击数(C系)、Perfect个数(P系)、得分(S系)、星标音符Perfect(SP系)，同一张卡在不同的Live以及手残程度不同的玩家手下，技能发动次数的期望也会大大不同。

技能强度是在Full Combo的前提下计算的，此外我们需要知道一些参数:
* 玩家队伍的强度 $S$
* 玩家的Perfect率 $\alpha$，即玩家每次点击以 $\alpha$ 的概率是Perfect，$(1-\alpha)$ 的概率是Great
* Live的时长 $T$ 秒，总Note个数 $N$，单点／长音符／滑键／星标音符占比 $ \rho_{单点}, \rho_{长音符}, \rho_{滑键}, \rho_{星标} $，满足 $ \rho_{单点} + \rho_{长音符} + \rho_{滑键} = 1 $
* 应援分数加成系数 $ \xi_{分} $，应援技能加成系数 $ \xi_{技} $，(当应援不存在时为 $1$，应援存在时游戏中固定为 $1.1$)

每个技能都是以发动条件个数 $m$， 触发概率 $p$， 奖励 $r$来描述的，例如“每达成29次连击，就有62%的概率提升分数290点”对应
$ m=29, p=0.62, r=290 $。
需要注意的是，在应援技能加成存在的条件下，需要将 $p$ 替换为 $p \times \xi_{技}$。

给定以上参数，我们可以进一步算出一些高级参数：

$ 【每秒\text{Note}个数】: \epsilon = N \,/\,T $

$ 【平均种类\&判定系数】: \phi = [\alpha \lambda(\text{Perfect}) + (1-\alpha) \lambda(\text{Great}) ] \times [\rho_{单点} \tau(单点) + \rho_{滑键} \tau(滑键)] + {[\alpha \lambda(\text{Perfect}) + (1-\alpha) \lambda(\text{Great}) ]}^2 \times \rho_{长音符} \tau(长音符) $

$ 【平均连击系数】: \bar{\chi} = \frac{1}{N} \sum_{i=1}^N \chi(i) $

$ 【平均每\text{Point}每\text{Note}对应的队伍强度】: \Delta = {[ \theta \times \phi \times \bar{\chi} ]}^{-1} $

$ 【每\text{Note}大致\text{Point}】: \omega = \xi_{分} \times S \, / \, \Delta $，这里暂时不考虑同色同团加成，后面会讨论同色同团加成对S系技能的加成

此外，需要注意在Live结尾是技能无法完全发动的情况，例如歌曲长度为110秒，技能为“每20秒以 $p$ 的概率提升分数 $r$点”，Live的最后十秒被浪费掉了，因此需要对技能的收益进行修正。

LL Helper的修正在随机余数均匀分布的条件下计算的，而FC前提下的N系、C系所有技能以及T系的非判定技能余数是固定的，本文对这些技能采用直接算余数的方法，与LL Helper的计算稍有不同。

注意：不要混淆【技能收益】和【技能强度】，技能收益是归一化后的技能奖励(见下文的【平均判定覆盖率】$G_{判}$、【每Note期望回复】$G_{奶}$、【每Note期望加分】$G_{分}$)，技能强度是技能收益换算后可以额外叠加在卡三围上的强度。
* 加分类技能有直接的技能强度 $ G_{分} \times \Delta $，在装备魅力宝石时为  $ 2.5 \times G_{分} \times \Delta $
* 回复类技能只有在装备治愈宝石后才有技能强度 $ 480 \times G_{奶} \times \Delta $
* 判定类技能在装备诡计宝石后才有技能强度，由于可能受益于卡组里其他判卡，在卡组确定计算总覆盖率之前无法准确计算其技能强度，但是根据单卡可以计算强度下限，如果装备了$\text{attr}$属性的诡计宝石，注意诡计宝石不受Center技能的加成，所以技能强度下限为$ 0.33 \times G_{判} \times【单卡界面前三围】(\text{attr}) $。

例如：总Note个数为700，长按比例为8%，滑键比例为4%，P率为95%，时间为120秒，星星为65个，队伍强度为80000，不存在应援的条件下

In [5]:
# 队伍强度，P率
S, alpha = 80000, 0.95
# Live时长，总Note个数
T, N = 120, 700
# 各种音符占比
rho_tap, rho_long, rho_swing, rho_star = 0.88, 0.08, 0.04, 65/700

epsilon = N/T
print('【每秒Note个数】', epsilon)
phi  = (alpha*lam_p+(1-alpha)*lam_g) * (rho_tap*tau_tap+rho_swing*tau_swing)
phi += (alpha*lam_p+(1-alpha)*lam_g)**2 * rho_long*tau_long
print('【平均种类&判定系数】', phi)
psi = np.array([chi(i) for i in range(1,N+1)]).mean()
print('【平均连击系数】', psi)
Delta = 1/(theta*phi*psi)
print('【平均每Point每Note对应的队伍强度】', Delta)
omega = S/Delta
print('【每Note大致Point】', omega)

【每秒Note个数】 5.833333333333333
【平均种类&判定系数】 1.2417545000000003
【平均连击系数】 1.2
【平均每Point每Note对应的队伍强度】 67.1093467616
【每Note大致Point】 1192.08432


需要注意的是，本文采用的计算方法【平均每Point每Note对应的队伍强度】要比LL Helper中的高，因为LL Helper中的技能强度在计算【每Note大致Point】时考虑了同色同团加成

In [6]:
print('LL Helper源码中平均每Point每Note对应的队伍强度', 80/1.1/1.1/1.2/(0.88+0.12*0.95))

LL Helper源码中平均每Point每Note对应的队伍强度 55.42899268891586


## 4.1 加强判定技能
对于判定系技能，我们以【平均判定覆盖率】$G_{判}$ 来衡量技能收益，需要注意的是所谓的【平均判定覆盖率】只是一个方便计算但非常粗略的估计，想要了解更多关于覆盖率的计算可以参考
[判定覆盖率和判卡培养](https://www.lovelivewiki.com/w/%E5%88%A4%E5%AE%9A%E8%A6%86%E7%9B%96%E7%8E%87%E5%92%8C%E5%88%A4%E5%8D%A1%E5%9F%B9%E5%85%BB)。

### 4.1.1 T判：每 $m$ 秒以 $p$ 的几率获得增强判定状态，该状态持续 $r$ 秒
对于T判，需要注意的是在判定加强期间技能CD不会走，因此【修正前平均判定覆盖率】为 
$$ G_{判}^\prime = \frac{pr/m}{1+pr/m} = \frac{pr}{m+pr} $$

在Live尾部可能的造成技能浪费的有两种情况
* 在最后一个Note结束时，判定强化状态仍然持续一段时间，损失为 

  $$ \mathbb{P}(最后一个\text{Note}结束时处于判定加强状态) \times \mathbb{E}[判定加强剩余时间] = G_{判}^\prime \times \frac{r}{2} \quad 秒 $$ 
  
* 上一次尝试触发技能的时刻到最后一个Note结束的时刻的间隔小于 $m$ 秒，损失为

  $$ \mathbb{P}(最后一个\text{Note}结束时不处于判定加强状态) \times \mathbb{E}[判定加强已冷却时间] = （1-G_{判}^\prime） \times \frac{m}{2} \quad 秒 $$ 
  
综上所述，修正后的【平均判定覆盖率】为

$$ G_{判} = \{G_{判}^\prime [T-(1-G_{判}^\prime) \times \frac{m}{2}] - G_{判}^\prime \times \frac{r}{2} \} \,/\, T 
  = G_{判}^\prime (1 - \frac{(1-G_{判}^\prime)m+r}{2T} ) $$

### 4.1.2 N判/C判：每 $m$ 个Note／连击，以 $p$ 的几率获得增强判定状态，该状态持续 $r$ 秒
由于覆盖自重合现象的存在，N判/C判【平均判定覆盖率】的计算需要分成两种情况考虑：

* $m > r \epsilon$：在这个情况下，$r$ 秒内不可能完成 $m$ 个Note/连击的计数，因此自重合现象不存在。

  $pr/m$ 为平均每Note判定增强状态时长，计算【修正前平均判定覆盖率】需要把“每Note”转换成“每秒”，即乘以【每秒Note个数】：
  $$ G_{判}^\prime = 【平均每\text{Note}判定增强状态时长】\times【每秒\text{Note}个数】 = \frac{pr}{m} \times \epsilon $$
  
* $m \le r \epsilon$：在这个情况下自重合现象有可能发生。如果不考虑自重合损失使用上面的公式的话在【每秒Note个数】的的Live下【平均判定覆盖率】会超过 $1$

  在每个覆盖区间开始时，累计继承状态为0，覆盖区间内技能满足触发条件的次数为 $ \frac{\epsilon r}{m} $。
  其中第一次满足触发条件时将累计继承状态置为1，在当前覆盖区间结束时将累计继承状态改回0并以 $p$ 的概率立即衔接一个新的覆盖区间。
  剩下的 $ \frac{\epsilon r}{m}-1 $ 次为自重合损失，因此每个覆盖区间内平均损失了 $ (\frac{\epsilon r}{m}-1)p $ 个覆盖区间。
  
  $$ G_{判}^\prime = 【平均每\text{Note}判定增强状态时长】\times【每秒\text{Note}个数】\times \frac{1}{ 1 + (\frac{\epsilon r}{m}-1)p }  = \frac{\epsilon pr}{\epsilon pr + (1-p)m} $$

综合以上两种情况，【修正前平均判定覆盖率】可以写为
$$ G_{判}^\prime = \frac{ \epsilon p r }{m + p\max(\epsilon r - m, 0)} $$

由于我们假设FC，因此给定总Note个数 $N$ 以后，可以算出技能做后一次尝试触发是在第 $ N^\prime = N - \mathsf{Mod}(N,m) $ 个Note处。

修正后的【平均判定覆盖率】为

$$ G_{判} = [G_{判}^\prime \times T - \frac{pr}{m} \times \mathsf{Mod}(N,m) ] \,/\, T 
  = G_{判}^\prime (1 - \frac{\mathsf{Mod}(N,m)}{N} ) $$

载入LLWiki 覆盖率计算页面数据

In [7]:
import urllib.request
url = 'https://www.lovelivewiki.com/w/%E5%88%A4%E5%AE%9A%E8%A6%86%E7%9B%96%E7%8E%87%E5%92%8C%E5%88%A4%E5%8D%A1%E5%9F%B9%E5%85%BB'
df_total = pd.read_html(urllib.request.urlopen(url))

In [8]:
get_pct = lambda x: '{0:.4f}%'.format(x*100)
df_list = []
for idx, N, T in zip([-3,-2,-1], [722, 584, 383], [110,110,85]):
	df = df_total[idx].iloc[2:]
	df.index = list(range(0,len(df)))
	# CR - cover rate, SCLR - self-covering loss rate
	df.columns = ['Card', 'note', 'prob', 'duration', 'CR', 'SCLR']
	df = pd.concat([df[['Card', 'note', 'prob', 'duration', 'CR']],
					pd.DataFrame(columns=['Approx CR'])], axis=1)
	epsilon = N/T
	# note_density = num_note/song_length
	for i, item in df.iterrows():
		m, p, r = int(item.note), float(item.prob)/100, float(item.duration)
		GJ0 = epsilon*p*r / (m+p*np.maximum(epsilon*r-m,0))
		GJ = (GJ0 * (1-np.mod(N,m)/N))
		df.iloc[i,-1] = '{0:.1f}%'.format(GJ*100)
	df_list.append(df)

人鱼1 Master 与精确计算的覆盖率对比

In [9]:
df_list[0].transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
Card,天女绘Lv8,天女绘Lv7,天女绘Lv6,天女绘Lv5,天女绘Lv4,魔术姬Lv8,魔术姬Lv7,魔术姬Lv6,魔术姬Lv5,魔术姬Lv4,魔术姬Lv3,吃瓜希Lv8,吃瓜希Lv7,吃瓜希Lv6,吃瓜希Lv5,捉奸鸟Lv8,捉奸鸟Lv7,捉奸鸟Lv6,捉奸鸟Lv5,捉奸鸟Lv4,旗袍妮Lv8,旗袍妮Lv7,旗袍妮Lv6,旗袍妮Lv5,花语花Lv8,花语花Lv7,花语花Lv6,花语花Lv5,花语花Lv4,花语花Lv3,红叶妮Lv8,红叶妮Lv7,红叶妮Lv6,红叶妮Lv5,红叶妮Lv4,旗手妮Lv8,旗手妮Lv7,旗手妮Lv6
note,17,17,17,17,17,15,15,15,15,15,15,32,32,32,32,23,23,23,23,23,32,32,32,32,17,17,17,17,17,17,16,16,16,16,16,31,31,31
prob,44,41,38,35,32,42,38,34,30,26,22,81,77,73,69,78,74,70,66,62,76,71,66,61,64,60,56,52,48,44,38,36,34,32,30,79,74,70
duration,9,8,8,7,7,9.5,9,9,8,8,7,8,7,7,6,7,6,6,5,5,9,8,8,7,6.5,6,6,5,5,4,7,6.5,6,5.5,5,6.5,6,5.5
CR,76.0%,70.0%,67.1%,62.3%,58.8%,76.7%,72.0%,68.2%,62.2%,57.0%,47.3%,87.9%,84.2%,81.5%,75.2%,88.7%,86.4%,84.0%,77.8%,74.4%,86.8%,82.0%,78.6%,72.1%,84.3%,80.8%,77.9%,71.8%,68.2%,58.8%,66.5%,63.4%,58.5%,53.6%,48.9%,85.3%,80.2%,74.2%
Approx CR,72.4%,67.5%,64.7%,58.6%,55.4%,74.9%,70.5%,66.8%,59.8%,55.0%,46.2%,85.3%,80.7%,77.5%,71.4%,86.5%,81.9%,79.0%,72.6%,69.1%,83.3%,78.1%,74.2%,67.5%,80.8%,76.8%,73.8%,66.9%,63.3%,54.2%,63.6%,59.8%,55.8%,51.4%,46.7%,82.8%,77.4%,72.2%


人鱼2 Expert 与精确计算的覆盖率对比

In [10]:
df_list[1].transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
Card,天女绘Lv8,天女绘Lv7,天女绘Lv6,天女绘Lv5,天女绘Lv4,魔术姬Lv8,魔术姬Lv7,魔术姬Lv6,魔术姬Lv5,魔术姬Lv4,魔术姬Lv3,吃瓜希Lv8,吃瓜希Lv7,吃瓜希Lv6,吃瓜希Lv5,捉奸鸟Lv8,捉奸鸟Lv7,捉奸鸟Lv6,捉奸鸟Lv5,捉奸鸟Lv4,旗袍妮Lv8,旗袍妮Lv7,旗袍妮Lv6,旗袍妮Lv5,花语花Lv8,花语花Lv7,花语花Lv6,花语花Lv5,花语花Lv4,花语花Lv3,红叶妮Lv8,红叶妮Lv7,红叶妮Lv6,红叶妮Lv5,红叶妮Lv4,旗手妮Lv8,旗手妮Lv7,旗手妮Lv6
note,17,17,17,17,17,15,15,15,15,15,15,32,32,32,32,23,23,23,23,23,32,32,32,32,17,17,17,17,17,17,16,16,16,16,16,31,31,31
prob,44,41,38,35,32,42,38,34,30,26,22,81,77,73,69,78,74,70,66,62,76,71,66,61,64,60,56,52,48,44,38,36,34,32,30,79,74,70
duration,9,8,8,7,7,9.5,9,9,8,8,7,8,7,7,6,7,6,6,5,5,9,8,8,7,6.5,6,6,5,5,4,7,6.5,6,5.5,5,6.5,6,5.5
CR,71.5%,66.8%,63.7%,56.7%,53.1%,73.8%,68.6%,64.4%,56.7%,51.4%,42.9%,85.0%,80.3%,76.9%,65.3%,87.1%,82.7%,79.7%,71.0%,67.2%,84.0%,78.0%,74.1%,65.8%,81.8%,78.3%,75.1%,68.0%,64.0%,52.7%,62.3%,58.1%,53.6%,49.7%,44.7%,79.8%,72.1%,63.6%
Approx CR,68.1%,62.8%,59.9%,53.5%,50.2%,69.2%,64.5%,60.6%,53.5%,48.7%,40.1%,83.8%,78.5%,74.8%,67.7%,83.8%,78.5%,75.2%,68.1%,64.3%,81.4%,75.4%,71.1%,63.6%,77.5%,73.0%,69.7%,62.2%,58.4%,49.0%,57.9%,54.1%,49.9%,45.6%,41.0%,77.1%,71.2%,63.0%


爱太阳 Expert 与精确计算的覆盖率对比

In [11]:
df_list[2].transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
Card,天女绘Lv8,天女绘Lv7,天女绘Lv6,天女绘Lv5,天女绘Lv4,魔术姬Lv8,魔术姬Lv7,魔术姬Lv6,魔术姬Lv5,魔术姬Lv4,魔术姬Lv3,吃瓜希Lv8,吃瓜希Lv7,吃瓜希Lv6,吃瓜希Lv5,捉奸鸟Lv8,捉奸鸟Lv7,捉奸鸟Lv6,捉奸鸟Lv5,捉奸鸟Lv4,旗袍妮Lv8,旗袍妮Lv7,旗袍妮Lv6,旗袍妮Lv5,花语花Lv8,花语花Lv7,花语花Lv6,花语花Lv5,花语花Lv4,花语花Lv3,红叶妮Lv8,红叶妮Lv7,红叶妮Lv6,红叶妮Lv5,红叶妮Lv4,旗手妮Lv8,旗手妮Lv7,旗手妮Lv6
note,17,17,17,17,17,15,15,15,15,15,15,32,32,32,32,23,23,23,23,23,32,32,32,32,17,17,17,17,17,17,16,16,16,16,16,31,31,31
prob,44,41,38,35,32,42,38,34,30,26,22,81,77,73,69,78,74,70,66,62,76,71,66,61,64,60,56,52,48,44,38,36,34,32,30,79,74,70
duration,9,8,8,7,7,9.5,9,9,8,8,7,8,7,7,6,7,6,6,5,5,9,8,8,7,6.5,6,6,5,5,4,7,6.5,6,5.5,5,6.5,6,5.5
CR,67.0%,62.0%,58.7%,51.9%,48.3%,69.1%,63.9%,59.6%,52.5%,47.1%,38.0%,78.5%,70.2%,66.8%,54.1%,83.0%,76.4%,73.0%,61.6%,57.9%,77.5%,70.4%,66.0%,56.1%,78.5%,74.3%,70.9%,61.8%,57.7%,45.2%,57.2%,53.5%,49.7%,44.3%,38.9%,69.6%,59.8%,52.1%
Approx CR,63.7%,58.2%,55.2%,48.8%,45.5%,66.0%,61.1%,57.0%,49.7%,44.8%,36.5%,76.1%,69.8%,66.1%,53.6%,79.7%,74.0%,70.4%,62.1%,58.4%,73.6%,67.4%,63.1%,55.3%,73.6%,68.8%,65.4%,57.6%,53.7%,44.4%,52.6%,48.7%,44.7%,40.5%,36.2%,72.5%,62.7%,54.4%


## 4.2 回复技能
对于回复系技能，我们以【每Note期望回复】$G_{奶}$ 来衡量技能收益

### 4.2.1 T奶：每 $m$ 秒，以 $p$ 的概率回复体力 $r$ 点

【修正前每Note期望回复】为 
$$ G_{奶}^\prime = 【每秒期望回复】/【每秒\text{Note}个数】= \frac{pr}{m} \,/\, \epsilon $$

修正后的【每Note期望回复】为 
$$ G_{奶} = [ G_{奶}^\prime \times N - \frac{pr}{m} \times \mathsf{Mod}(T,m) ]/N
  = G_{奶}^\prime (1-\frac{\mathsf{Mod}(T,m)}{T}) $$

### 4.2.2 N奶/C奶：每 $m$ 个Note／连击，以 $p$ 的概率回复体力 $r$ 点

【修正前每Note期望回复】为 
$$ G_{奶}^\prime = \frac{pr}{m} $$

修正后的【每Note期望回复】为 
$$ G_{奶} = G_{奶}^\prime (1-\frac{\mathsf{Mod}(N,m)}{N}) $$

### 4.2.3 P奶：每 $m$ 个Perfect，以 $p$ 的概率回复体力 $r$ 点

【修正前每Note期望回复】为 
$$ G_{奶}^\prime = 【每\text{Perfect}期望回复】\times 【\text{Perfect}率】= \frac{pr}{m} \times \alpha $$

修正后的【每Note期望回复】为 
$$ G_{奶} = G_{奶}^\prime (1-\frac{m/2}{\alpha N}) = G_{奶}^\prime (1-\frac{m}{2\alpha N}) $$

## 4.3 加分技能
对于加分系技能，我们以【每Note期望加分】$G_{分}$ 来衡量技能收益

### 4.3.1 T分：每 $m$ 秒，以 $p$ 的概率提升 $r$ 分

【修正前每Note期望加分】为 
$$ G_{分}^\prime = 【每秒期望加分】/【每秒\text{Note}个数】= \frac{pr}{m} \,/\, \epsilon $$

修正后的【每Note期望加分】为 
$$ G_{分} = [ G_{分}^\prime \times N - \frac{pr}{m} \times \mathsf{Mod}(T,m) ]/N
  = G_{分}^\prime (1-\frac{\mathsf{Mod}(T,m)}{T}) $$

### 4.3.2 N分/C分：每 $m$ 个Note／连击，以 $p$ 的概率提升 $r$ 分

【修正前每Note期望加分】为 
$$ G_{分}^\prime = \frac{pr}{m} $$

修正后的【每Note期望加分】为 
$$ G_{分} = G_{分}^\prime (1-\frac{\mathsf{Mod}(N,m)}{N}) $$

### 4.3.3 P分：每 $m$ 个Perfect，以 $p$ 的概率提升 $r$ 分

【修正前每Note期望加分】为 
$$ G_{分}^\prime = 【每\text{Perfect}期望加分】\times 【\text{Perfect}率】= \frac{pr}{m} \times \alpha $$

修正后的【每Note期望加分】为 
$$ G_{分} = G_{分}^\prime (1-\frac{m/2}{\alpha N}) = G_{分}^\prime (1-\frac{m}{2\alpha N}) $$

### 4.3.4 S分：每当Point到达 $m$ 的倍数，以 $p$ 的概率提升 $r$ 分
由于S系技能依赖于全队得分在Live中的平均能力，具体见Section 7.4对【加权平均位置加成】$ \bar{\mu} $ 的定义。
比如如果队里的卡全都与在打的Live同色同团，得分大致会有$ \mu_{团} \times \mu_{色} $倍(即$1.1$倍)的加成，S系技能的尝试触发次数也会增多。

大多数情况下我们卡组中的卡都是和Live同色的，假设有一半的卡和Live同团，Live谱面各个位置权重相同，粗算时我们可以假设 $ \bar{\mu}=1.15 $。
需要注意一次技能发动后分数增加会促进下一次技能发动，因此期望上技能每 $m-pr$ 分就会发动一次。

【修正前每Note期望加分】为 
$$ G_{分}^\prime = 【每\text{Point}期望加分】\times (【加权平均位置加成】\times【每\text{Note}大致\text{Point}】)= \frac{pr}{m-pr} \times \bar{\mu} \omega $$

修正后的【每Note期望加分】为 
$$ G_{分} = G_{分}^\prime (1-\frac{m/2}{\bar{\mu} \omega N}) = G_{分}^\prime (1-\frac{m}{2\bar{\mu} \omega N}) $$

### 4.3.5 SP分：每 $m$ 个星星音符 Perfect，以 $p$ 的概率提升 $r$ 分

【修正前每Note期望加分】为 
$$ G_{分}^\prime = 【每星星音符\text{Perfect}期望加分】\times【星标音符占比】\times【\text{Perfect}率】】= \frac{pr}{m} \times \rho_{\mathrm{star}} \times \alpha $$

修正后的【每Note期望加分】为 
$$ G_{分} = G_{分}^\prime (1-\frac{m}{2\alpha \, \rho_{\mathrm{star}} \, N}) $$

注：由于游戏中只有一张SP判“每获得1个星星音符Perfect，以26%的几率增加120积分”，因此这张卡无需修正可以直接用 $G_{分}^\prime$ 来表示其期望收益。

# 5 LL Helper通用强度的计算方式
几点前提知识
* 引入【平均C位加成】$\bar{c}$ 的概念
  $$ \bar{c}= 1 + \sum_{\text{card} \in 非特典\text{UR}} (\text{card}的主\text{C}加成+\text{card}的副\text{C}加成) \,\,/ 【非特典\text{UR}个数】 $$
  到2017年6月17日为止，$ \bar{c} = 1.15 $。

* 为了方便表述，如果一个宝石槽被技能宝石(魅力、治愈、诡计)占用则被称为【技能宝石槽】，反之则为【非技能宝石槽】。

  忽略比较弱的增加数值的宝石(吻和香水)，我们来看一些非技能宝石的每槽加成效果，假设我们的卡组由九张相同的卡组成：
  * 指环宝石消耗2槽，单体加成$10\%$，每槽$10\% /2 = 5\%$
  * 十字宝石消耗3槽，单体加成$16\%$，每槽$16\% /3 = 5.33\%$
  * 光环宝石消耗3槽，团体加成$1.8\%$，每槽$1.8\% \times 9 \,/\, 3 = 5.4\%$
  * 面纱宝石消耗4槽，团体加成$2.4\%$，每槽$2.4\% \times 9 \,/\, 4 = 5.4\%$

  LL Helper按照每槽$5.2\%$来估计每个【非技能宝石槽】。
* 回顾之前对于技能强度的计算
  
  * 加分类技能有直接的技能强度 $ G_{分} \times \Delta $，在装备魅力宝石时为  $ 2.5 \times G_{分} \times \Delta $，对分卡设
  
    $$ 【技能强度】(0) = G_{分} \times \Delta, \qquad 【技能强度】(1) = 2.5 \times G_{分} \times \Delta $$
  * 回复类技能只有在装备治愈宝石后才有技能强度 $ 480 \times G_{奶} \times \Delta $，对奶卡设
  
    $$ 【技能强度】(0) = 0, \qquad 【技能强度】(1) = 480 \times G_{奶} \times \Delta $$
  * 判定类技能在装备诡计宝石后才有技能强度，由于可能受益于卡组里其他判卡，在卡组确定计算总覆盖率之前无法准确计算其技能强度，但是根据单卡可以计算强度下限，如果装备了$\text{attr}$属性的诡计宝石，那么技能强度下限为$ 0.33 \times G_{判} \times【团队真实三围】(\text{attr}) $。
    
    对判卡，LL Helper在计算通用强度时不考虑判定技能带来的技能强度，即
    $$ 【技能强度】(0) = 0, \qquad 【技能强度】(1) = 0 $$
    但是笔者认为可以改为
    $$ 【技能强度】(0) = 0, \qquad 【技能强度】(1) = 0.33 \times G_{判} \times【含绊三围】(\text{attr}) \times (1+5.2\% \times (【宝石槽个数】-4)) $$

阅读LL Helper代码后，笔者理解（可能理解有误）的卡 $\text{card}$ 属性 $\text{attr}$ 通用强度为该卡在打同团 $\text{attr}$ 时的Live时通过装备技能宝石可以达到的最强数值除以 $ \mu_{团} \times \mu_{色} $。
具体来讲

$$ 【通用强度】(\text{attr}) = \frac{1}{ \mu_{团} \mu_{色} } {\large\{}  \mu_{团} \mu_{色}^{\mathbb{I} (\text{attr}=主属性)} \quad \bar{c} \times 【含绊三围】(\text{attr}) \times (1 + 5.2\% \times【非技能宝石槽个数】) +【技能强度】(\mathbb{I}(装备技能宝石)){\large\}} $$

首先，定义【修正三围】为
$$ 【修正三围】(\text{attr}) = \bar{c} \times 【含绊三围】(\text{attr}) \,\,/\,\, \mu_{色}^{\mathbb{I}(\text{attr} \ne 主属性)} $$



对于槽数大于等于4的卡，需要考虑是否花费四个宝石槽装备技能宝石。
给定一张宝石槽数为 $s$ 的卡，若 $ s \ge 4 $，那么选定一种属性，我们可以用同属性的非技能宝石填满4个槽得到【标准四槽强度】：
 
$$ 【标准四槽强度】(\text{attr}) = \mu_{团} \mu_{色} \times【修正三围】(\text{attr}) \times 0.052 \times 4 = 0.208 \, \mu_{团} \mu_{色} \times【修正三围】(\text{attr}) $$

## 5.1 判卡或宝石槽数小于4的情况
当宝石槽数小于4时无法装备技能宝石，当卡是判卡的时候如上文所说在整个卡组确定前没有很好的估算技能强度的方法，故LL Helper暂时不考虑诡计宝石，因此
$$ 【通用强度】(\text{attr}) = 【修正三围】(\text{attr}) \times (1+0.052s) $$

举例：
![](Gereal_Strength_Judge.png)

In [12]:
# 计算【含绊三围】
base_attr = np.array([4570, 4030, 5320])
bond = np.array([0, 0, 1000])
base_bond_attr = base_attr + bond
# 计算【修正三围】
amend_attr = 1.15 * base_bond_attr / 1.1**(bond==0)
print('【修正三围】', np.round(amend_attr))
# 计算【通用强度】
general_strength = np.floor(amend_attr * (1+0.052*5))
print('【通用强度】', general_strength)

【修正三围】 [ 4778.  4213.  7268.]
【通用强度】 [ 6019.  5308.  9157.]


如果按照笔者提出的估计方法$ 0.33 \times G_{判} \times【团队\text{C}前三围】(\text{attr}) $

In [17]:
# 计算【含绊三围】
base_attr = np.array([4570, 4030, 5320])
bond = np.array([0, 0, 1000])
base_bond_attr = base_attr + bond
# 计算【修正三围】
amend_attr = 1.15 * base_bond_attr / 1.1**(bond==0)
print('【修正三围】', np.round(amend_attr))
# 计算【技能强度】
m, p, r = 25, 0.71, 7
gain = p*r/m / (1+p*r/m)
gain = gain * (1 - ((1-gain)*m+r)/(2*T))
print('【平均判定覆盖率】', gain)
skill_strength = 0.33 * gain * (1+0.052*1) * amend_attr
print('【技能强度】', np.round(skill_strength))
# 计算【通用强度】分支1
branch1 = np.floor(amend_attr * (1+0.052*1) + skill_strength/(mu_g*mu_c) )
print('【通用强度】分支1: 装备诡计宝石  ', branch1)
# 计算【通用强度】分支2
branch2 = np.floor(amend_attr * (1+0.052*5))
print('【通用强度】分支2: 不装备诡计宝石', branch2)

print('【通用强度】', np.maximum(branch1, branch2))

【修正三围】 [ 4778.  4213.  7268.]
【平均判定覆盖率】 0.1386611489963208
【技能强度】 [ 230.  203.  350.]
【通用强度】分支1: 装备诡计宝石   [ 5216.  4599.  7935.]
【通用强度】分支2: 不装备诡计宝石 [ 6019.  5308.  9157.]
【通用强度】 [ 6019.  5308.  9157.]


## 5.2 奶卡且宝石槽数至少为4的情况
如果装备了治愈宝石，其对应强度为 $ 【治愈宝石强度】 = 480 \times G_{\mathrm{G}} \times \Delta $

对于每一个属性$ \text{attr} $， 我们对比 $【治愈宝石强度】$ 和 $【标准四槽强度】(\text{attr})$
* 如果 $【治愈宝石强度】(\text{attr}) > 【标准四槽强度】(\text{attr})$，那么花费4槽装备治愈宝石，然后用非技能宝石填满剩下的槽
  $$ 【通用强度】(\text{attr}) = 【修正三围】(\text{attr}) \times [1+0.052(s-4)] + \frac{480 \times G_{\mathrm{G}} \times \Delta}{\mu_{团} \mu_{色}} $$
* 如果 $【治愈宝石强度】(\text{attr}) \le【标准四槽强度】(\text{attr})$，那么用非技能宝石填满所有的槽
  $$ 【通用强度】(\text{attr}) = 【修正三围】(\text{attr}) \times (1+0.052s) $$
  
举例：
![](Gereal_Strength_Stamina.png)

In [14]:
# 计算【含绊三围】
base_attr = np.array([4400, 5210, 4180])
bond = np.array([0, 1000, 0])
base_bond_attr = base_attr + bond
# 计算【修正三围】
amend_attr = 1.15 * base_bond_attr / 1.1**(bond==0)
print('【修正三围】', np.round(amend_attr))
# 计算【技能强度】
m, p, r = 14, 0.64, 7
gain = p*r/m / epsilon * (1-epsilon*np.mod(T,m)/N)
print('【每Note期望回复】', gain)
skill_strength = 480 * Delta * gain
print('【技能强度】', np.round(skill_strength))
# 计算【通用强度】分支1
branch1 = np.floor(amend_attr * (1+0.052*2) + skill_strength/(mu_g*mu_c) )
print('【通用强度】分支1: 装备治愈宝石  ', branch1)
# 计算【通用强度】分支2
branch2 = np.floor(amend_attr * (1+0.052*6))
print('【通用强度】分支2: 不装备治愈宝石', branch2)

print('【通用强度】', np.maximum(branch1, branch2))

【修正三围】 [ 4600.  7141.  4370.]
【每Note期望回复】 0.070182767624
【技能强度】 2261.0
【通用强度】分支1: 装备治愈宝石   [ 6946.  9752.  6692.]
【通用强度】分支2: 不装备治愈宝石 [ 6035.  9369.  5733.]
【通用强度】 [ 6946.  9752.  6692.]


## 5.3 分卡且宝石槽数至少为4的情况
如果装备了魅力宝石加分技能的收益提升了 $2.5-1=1.5$ 倍，其对应强度为 $ 【魅力宝石强度】 = 1.5 \times G_{\mathrm{S}} \times \Delta $

对于每一个属性$ \text{attr} $， 我们对比 $【魅力宝石强度】$ 和 $【标准四槽强度】(\text{attr})$
* 如果 $【魅力宝石强度】(\text{attr}) >【标准四槽强度】(\text{attr})$，那么花费4槽装备魅力宝石，然后用非技能宝石填满剩下的槽
  $$ 【通用强度】(\text{attr}) = 【修正三围】(\text{attr}) \times [1+0.052(s-4)] + \frac{2.5 \times G_{\mathrm{S}} \times \Delta}{\mu_{团} \mu_{色}} $$
* 如果 $【魅力宝石强度】(\text{attr}) \le【标准四槽强度】(\text{attr})$，那么用非技能宝石填满所有的槽
  $$ 【通用强度】(\text{attr}) = 【修正三围】(\text{attr}) \times (1+0.052s) + \frac{G_{\mathrm{S}} \times \Delta}{\mu_{团} \mu_{色}} $$
  
举例：
![](Gereal_Strength_Score.png)

In [15]:
# 计算【含绊三围】
base_attr = np.array([5330,4110,4470])
bond = np.array([1000, 0, 0])
base_bond_attr = base_attr + bond
# 计算【修正三围】
amend_attr = 1.15 * base_bond_attr / 1.1**(bond==0)
print('【修正三围】', np.round(amend_attr))
# 计算【技能强度】
m, p, r = 21, 0.39, 1280
gain = p*r/m * (1 - np.mod(N,m)/N)
print('【每Note期望加分】', gain)
skill_strength = gain * Delta
print('【技能强度】', np.round(skill_strength))
# 计算【通用强度】分支1
branch1 = np.floor(amend_attr * (1+0.052*1) + 2.5*skill_strength/(mu_g*mu_c) )
print('【通用强度】分支1: 装备魅力宝石  ', branch1)
# 计算【通用强度】分支2
branch2 = np.floor(amend_attr * (1+0.052*5) + skill_strength/(mu_g*mu_c) )
print('【通用强度】分支2: 不装备魅力宝石', branch2)

print('【通用强度】', np.maximum(branch1, branch2))

【修正三围】 [ 7279.  4297.  4673.]
【每Note期望加分】 23.4610966057
【技能强度】 1574.0
【通用强度】分支1: 装备魅力宝石   [ 10911.   7773.   8169.]
【通用强度】分支2: 不装备魅力宝石 [ 10473.   6715.   7189.]
【通用强度】 [ 10911.   7773.   8169.]


# 6 谱面进阶数据
谱面进阶数据是在Full Combo以及预设【Perfect率】$\alpha$ 的前提下计算的，谱面中的每一个Note都可以用 $ (\text{index}, \text{type}, \text{pos}, \text{time}) $ 来表示，其中$ \text{time} $ 指的是通过Note结束时间，对于长音符这等于Note起始时间加长音符存在时长，
$ \text{index} $ 是通过Note结束时间 $\text{time}$ 来排序的，比如Live开始第一个Note是长音符，但是这个长音符结束前还有三个单点音符，那么这个长音符的 $\text{index}$ 就是4。

首先引入几个概念：
* 【总Note个数】$N$：谱面Note总数
* 【Live时长】$T$：指的是从歌曲封面出现到最后一个Note结束时所经过的时间，是时间系技能可以触发的时间量
* 【歌曲时长】$T_\star$：指的是从歌曲封面出现到“Live成功”字样出现时所经过的时间，要略长于【Live时长】
* 【Combo乘数】$\psi$：指的是Full Combo比起Max Combo不到50时的得分的倍数，实际就是上文的【平均连击系数】
  $$ \psi = \frac{1}{N} \sum_{i=1}^N \chi(i) $$
* 【图标权重】$w_{\mathrm{note}}$：即Note的种类系数
  $$ w_{\mathrm{note}}(\text{Note}) = \tau(\text{type}) $$
* 【连击权重】$w_{\mathrm{combo}}$：一个Note在给定P率下期望的 连击系数、判定系数和种类系数的乘积
  $$ w_{\mathrm{combo}}(\text{Note}) = \chi(\text{index}) \times [\alpha \lambda(\text{Perfect}) + (1-\alpha) \lambda(\text{Great}) ]^{ 1 + \mathbb{I}(\text{type}=长音符) } \quad \times w_{\mathrm{note}}(\text{Note}) $$
* 【位置图标/连击权重】$ W_{\mathrm{note}}, W_{\mathrm{combo}} $ , 【总图标/连击权重】$ \mathbf{W}_{\mathrm{note}}, \mathbf{W}_{\mathrm{combo}} $，【位置连击权重占比】$\zeta$：
  $$ W_{\mathrm{note}}(位置) = \sum_{所有\text{Note}} w_{\mathrm{note}}(\text{Note}) \times \mathbb{I}(\text{pos}=位置), \qquad
     \mathbf{W}_{\mathrm{note}} = \sum_{k=1}^9 W_{\mathrm{note}}(位置) \\
     W_{\mathrm{combo}}(位置) = \sum_{所有\text{Note}} w_{\mathrm{combo}}(\text{Note}) \times \mathbb{I}(\text{pos}=位置), \quad
     \mathbf{W}_{\mathrm{combo}} = \sum_{k=1}^9 W_{\mathrm{combo}}(位置), \quad
     \zeta(位置) = W_{\mathrm{combo}}(位置) \,\,/\,\, \mathbf{W}_{\mathrm{combo}}
  $$ 
* 【单位强度得分】$\gamma$：(注：该定义与LL Helper数值不符，目前LL Helper单位强度得分计算方法未知) 在不考虑同色同团的条件下，每一点队伍强度可以得到的歌曲分数
  $$ \gamma = \theta \times \mathbf{W}_{\mathrm{combo}} $$


我们以Love wing bell的Master难度为例，需要注意的是计算结果与SIFdbV以及LL Helper有所不同，根本原因依然是采用的参数体系不同

In [16]:
import json
# 加载谱面json文件
temp = json.loads(open('Love wing bell Master.json').read())
df = pd.DataFrame(temp, index=list(range(1,len(temp)+1)))
# 确定每个note的种类
df = df.assign(token=df.effect==2, long=df.effect.apply(lambda x: x == 3), 
			   star=df.effect==4, swing=df.effect.apply(lambda x: x in [11,13]))
df['tap'] = df.apply(lambda x: not (x.long or x.swing), axis=1)
# 计算note的结束时间并按之排序
df.timing_sec = df.timing_sec + df.effect_value * df.long
df = df.sort_values(by='timing_sec', ascending=True)
df.index = [i for i in range(1, len(df)+1)]
# 计算每个note的【图标权重】和【连击权重】
alpha, lam_p, lam_g = 0.95, 1.25, 1.1
tau_long, tau_swing = 1, 0.5
df['图标权重'] = df.apply(lambda x: tau_long**x.long * tau_swing**x.swing, axis=1)
df['连击权重'] = df['图标权重'] * df.long.apply(lambda x: ( alpha*lam_p + (1-alpha)*lam_g )**(1+x) )\
					 * [chi(i) for i in range(1,len(df)+1)]
# 对每个位置进行统计
note_stat = df.groupby(by='position')[['tap', 'long', 'swing', 'star', 'token']].sum().applymap(int)
note_stat['图标权重'] = df.groupby(by='position')['图标权重'].sum()
note_stat['连击权重'] = df.groupby(by='position')['连击权重'].sum()
note_stat = note_stat.append(pd.DataFrame(note_stat.sum(), columns=['总和']).transpose())
note_stat['连击权重占比'] = note_stat['连击权重'] / note_stat.loc['总和','连击权重']
# 按照逆时针顺序计算位置
pos_name = ['L4', 'L3', 'L2', 'L1', 'C', 'R1', 'R2', 'R3', 'R4']
note_stat.index = [pos_name[9-x] if type(x)==int else x for x in list(note_stat.index)]
note_stat = note_stat.loc[pos_name+['总和']]
print('【Live时长】', df.iloc[-1].timing_sec, '秒')
print('【Combo乘数】', np.array([chi(i) for i in range(1,len(df)+1)]).mean())
print('【单位强度得分】', theta*note_stat.loc['总和','连击权重'])
note_stat.applymap(lambda x: str(int(x)) if np.isclose(x,round(x)) else '{0:.3f}'.format(x)).transpose()

【Live时长】 98.636 秒
【Combo乘数】 1.18636363636
【单位强度得分】 8.13246225312


,L4,L3,L2,L1,C,R1,R2,R3,R4,总和
tap,46,42,44,46,26,46,46,45,45,386
long,9,7,11,8,0,7,10,7,8,67
swing,11,15,20,17,24,19,23,19,15,163
star,13,4,5,9,1,9,7,5,17,70
token,3,3,1,2,2,3,4,5,4,27
图标权重,60.500,56.500,65,62.500,38,62.500,67.500,61.500,60.500,534.500
连击权重,92.002,85.719,100.087,95.723,55.757,95.363,103.468,93.454,91.672,813.246
连击权重占比,0.113,0.105,0.123,0.118,0.069,0.117,0.127,0.115,0.113,1


# 7 在给定谱面、卡组以及附加条件下估算总得分
在Full Combo以及预设【Perfect率】$\alpha_0$ 的前提下，当Live谱面和卡组以及附加条件(好友应援Center技能、应援得分/技能加成)确定时，我们可以计算
* 【团队强度】和【判定时团队强度】：见Section2、Section3，设与Live同属性的强度
  $$ S_0 = 【团队强度】(\text{live attr}), \qquad S_1 = 【判定时团队强度】(\text{live attr}) $$
* 【同色同团加成系数】$\mu$：见Section3
* 【每张判卡的技能收益】$G_{判}$：见Section4
* 【位置连击权重占比】$\zeta$ 和【单位强度得分】$\gamma$：见Section6

本文估算总得分的步骤分为如下几个Subsection。

## 7.1 团队期望覆盖率的粗略估计
Section4中提到，由于可能受益于卡组里其他判卡，在卡组确定计算总覆盖率之前无法计算其技能强度。
当Live谱面和卡组确定时，我们可以计算每张判卡的【平均判定覆盖率】$G_{判}$，并且假设每个Note到达时，处于判定加强状态的概率是独立同分布服从 $ \mathsf{Bernoilli}(G_{判}) $ 的。

由于不同判卡之间技能的发动是独立的，因此在当前假设下，【团队期望覆盖率】$ \mathrm{CR} $ 可以近似为
$$ \mathrm{CR} = 1 - \prod_{\text{card} \in 卡组中的判卡} ( 1 - \text{card} 的 G_{判} ) $$

## 7.2 修正Perfect率及团队强度
由于判卡以及诡计宝石的存在，游戏时最终得到的Perfect率和团队强度会因为判定强化状态而有所偏差，因此我们引入
$$ 【修正\text{Perfect}率】\alpha = 1 - (1-\alpha_0)(1-\mathrm{CR}) $$
$$ 【修正团队强度】S_{队} = S_0 \times (1-\mathrm{CR}) + S_1 \times \mathrm{CR} $$
注：实际上当【Perfect率】变化后，Live对应的【单位强度得分】也会改变，但是由于变化不大这里并不对其进行修正。

## 7.3 总回复&加分技能强度
使用【修正Perfect率】$\alpha$ 和【修正团队强度】$S$ 来计算回复和加分技能的收益，并且计算【总回复&加分技能强度】$ S_{技} $
$$ S_{技} = {\large\{} \sum_{\text{card} \in 卡组中的分卡} \text{card} 的 G_{分} \times {2.5}^{\mathbb{I}(\text{card}装备有魅力宝石)} \qquad + \sum_{\text{card} \in 卡组中的奶卡} \text{card} 的 G_{奶} \times 480 \times \mathbb{I}(\text{card}装备有治愈宝石) {\large\}} \times \Delta $$


## 7.4 期望总得分
由于不同位置的同色同团加成以及权重占比不同，我们需要计算【加权平均位置加成】$ \bar{\mu} = \sum_{i=1}^9 \mu(i) \zeta(i) $。
于是，【期望总得分】$\Omega$ 可以计算为为

\begin{align}
  \Omega &= 【通过点击获得的得分】+【通过技能获得的得分】\\
  &=【修正后团队强度】 \times 【单位强度得分】\times 【加权平均位置加成】\times 【应援分数加成系数】\\
  &\quad +【总回复\&加分技能强度】\times 【单位强度得分】\\
  &= \,\, S_{队} \times \gamma \times \bar{\mu} \times \xi_{分} + S_{技} \times \gamma
\end{align}

验证：依然使用本节开始例子，test_unit.sd／test_unit.ieb里的卡组，不考虑应援分数加成、应援技能加成以及好友应援Center技能

LL Helper卡组强度计算结果：期望得分只有102万（这里是笔者的乌龙，LL Helper里的三围是【含绊三围】，笔者导入了【基础三围】）
<img src="result_LLHelper.png" width="400">
ieb数据站模拟结果分布：得分主要分布在120万左右
<img src="result_ieb.png" width="800">
笔者模拟程序运行一次得到的结果，以及总分估算结果：单次模拟得分在120万左右，总分估算结果也是120万左右
<img src="result_me.png" width="800">

# 8 在给定谱面以及C位技能的条件下搜索最优卡组及宝石分配方案
首先引入【大致强度】的概念：【大致强度】是给定谱面、C位技能以及应援加成之后的【通用强度】，即考虑 $ \bar{c}, \mu_{团}, \mu_{色} $ 以及技能强度时代入已知条件。

由于C位技能已经给定，我们可以计算所有拥有该技能的卡的【大致强度】，并且选择最强的一张作为代表改C位技能的【C位卡】。
对于【C位卡】以外的所有卡，计算其【大致强度】，取前 $K$ 张作为【预备卡组】，LL组卡器里 $K$ 一般取 $15 \sim 20$。

我们的目标是给定【C位卡】，从【预备卡组】中找出另外8张卡组成卡组，并优化位置排布和宝石分配。
一旦我们可以做到这一点，只要再在外层对不同的C位技能套一层循环就可以找到最优卡组了。

## 8.1 计算单张卡宝石分配方案带来的额外得分
在给定【C为卡】、另外8张卡以及谱面，但是并没有确定每张非C位卡的位置时，首先我们可以计算【团队期望覆盖率】$\mathrm{CR}$，然后需要找到一个最大化【加权平均位置加成】 $\bar{\mu}$ 的位置排布。
注意到团队位置加成系数只有1, 1.1, 1.21三种可能性，我们只需把加成少的卡尽量往【位置连击权重占比】的位置放即可。

接下来，对每一张卡的每一种可能的宝石分配，计算宝石带来的额外得分。

对于每张卡我们可以列举它所有可能装备的技能宝石组合（包括所有同色宝石和技能宝石，不考虑异色宝石，虽然异色宝石可能会通过异色Center技带来额外得分，但是大多数情况下比同色差很多，考虑它只会大大增加计算量），比如一个有5个技能槽的分卡可能的组合有：

魅力+吻，诡计+吻，面纱+吻，光环+香水，光环+指环，十字+香水，十字+指环，指环+香水+吻，魅力，诡计，面纱，光环+吻，十字+吻，指环+香水，光环，十字，香水+吻，指环+吻，香水，指环，吻

根据之前对期望得分的计算公式，我们可以把所有项都展开，由于只有同色宝石，整个公式可以写成：
\begin{align}
  &【总期望得分】\\
  &= \gamma \times \bar{\mu} \times \chi_{分} \times {\huge\{} \sum_{i=1}^9  {\Large\{} {\large[} 卡i的【含绊三围】(\text{Live}属性) \times (1 + 【指环十字加成】+【光环面纱加成】) + 【吻香水增加】{\large]} \\
  &\quad \times (1+【\text{Live}属性 \text{Center}技加成】) + \sum_{其他属性} 卡i的【含绊三围】(其他属性) \times 【其他属性 \text{Center}技加成】\\
  & \quad + {\large[} 卡i的【含绊三围】(\text{Live}属性) \times (1 + 【指环十字加成】) + 【吻香水增加】{\large]} \times \mathbb{I}(卡i装备有诡计宝石) \times \text{CR} \times 33\% {\Large\}} {\huge\}} \\
  &\quad + \gamma \times {\large\{} \sum_{\text{card} \in 卡组中的分卡} \text{card} 的 G_{分} \times {2.5}^{\mathbb{I}(\text{card}装备有魅力宝石)} \qquad + \sum_{\text{card} \in 卡组中的奶卡} \text{card} 的 G_{奶} \times 480 \times \mathbb{I}(\text{card}装备有治愈宝石) {\large\}} \times \Delta
\end{align}
其中每张分卡奶卡的 $G_{分},G_{奶}$ 都是在修正后的Perfect率及团队强度下计算的。

需要注意的是，上面公式里单体宝石和诡计宝石有相乘的项，这里笔者将相乘的项的部分全部算作诡计宝石需要计算的部分，因此在计算单体宝石的额外的分时不考虑这部分。
于是，固定一张卡 $\text{card}$， 对于每种宝石，它们对应的额外得分为
$$ S_{吻} = 200 \times (1+【\text{Live}属性 \text{Center}技加成】) \times \gamma \times \bar{\mu} \times \chi_{分} $$
$$ S_{香水} = 450 \times (1+【\text{Live}属性 \text{Center}技加成】) \times \gamma \times \bar{\mu} \times \chi_{分} $$
$$ S_{指环} = 0.1 \times \text{card}的【含绊三围】(\text{Live}属性) \times (1+【\text{Live}属性 \text{Center}技加成】) \times \gamma \times \bar{\mu} \times \chi_{分} $$
$$ S_{十字} = 0.16 \times \text{card}的【含绊三围】(\text{Live}属性) \times (1+【\text{Live}属性 \text{Center}技加成】) \times \gamma \times \bar{\mu} \times \chi_{分} $$
$$ S_{光环} = 0.018 \times \sum_{i=1}^9 \text{card}的【含绊三围】(\text{Live}属性) \times (1+【\text{Live}属性 \text{Center}技加成】) \times \gamma \times \bar{\mu} \times \chi_{分} $$
$$ S_{面纱} = 0.024 \times \sum_{i=1}^9 \text{card}的【含绊三围】(\text{Live}属性) \times (1+【\text{Live}属性 \text{Center}技加成】) \times \gamma \times \bar{\mu} \times \chi_{分} $$
$$ S_{魅力} = \text{card} 的 G_{分} \times 1.5 \times \Delta \times \gamma $$ 
$$ S_{治愈} = \text{card} 的 G_{奶} \times 480 \times \Delta \times \gamma $$ 

任何不包含诡计宝石的分配方案都可以由以上公式相加而得，对于诡计宝石，需要考虑单体宝石对其的加成：
$$ S_{诡计}(单体宝石) =  {\large[} \text{card}的【含绊三围】(\text{Live}属性) \times (1 + 【指环十字加成】) + 【吻香水增加】{\large]}  \times \text{CR} \times 33\% \times \gamma \times \bar{\mu} \times \chi_{分} \\
\qquad = 33\% \times \mathrm{CR} \times {\large\{} \text{card}的【含绊三围】(\text{Live}属性) \times \gamma \times \bar{\mu} \times \chi_{分} + {(1+【\text{Live}属性 \text{Center}技加成】) \qquad }^{-1} \sum_{\text{gem} \in \text{card}装备的单体宝石} \qquad S_{\text{gem}} {\large\}} $$

## 8.2 在给定谱面和卡组的条件下进行最佳宝石分配
这里我们讨论如下问题：假设已知谱面和每种宝石的持有量，固定九张卡以及每张卡的位置，找出最佳宝石分配方案。

首先对每张卡的对每种组合我们都可以按照Section 8.1计算宝石所额外带来得分并按之降序排序。

设 $ \mathbb{G}_i $ 第 $i$ 张卡的【宝石组合天梯】，其中$ \mathbb{G}_{i,l} = (第l强的宝石组合，第l强的宝石组合额外带来的强度) $。
那么每种宝石分配都可以用一个长度为9的向量来表示：
$$ \mathbf{n} \triangleq [n_1, \ldots, n_9] \in \mathcal{O} \triangleq \bigotimes_{i=1}^9 \{ 1, 2, \ldots, | \mathbb{G}_i | \} $$

对于两个宝石分配方案 $ \mathbf{x}, \mathbf{y} $，记
$$ \mathbf{x} \preceq \mathbf{y} \quad \iff \quad x_i \le y_i, \quad \forall \, i \in \{1,\ldots,9\} $$
对于宝石分配方案 $ \mathbf{x} $， 定义其“父亲”、“儿子”、“祖先”、“子孙”集合、为
$$ \mathsf{Pa}(\mathbf{x}) = \left\{ \mathbf{z} \in \mathcal{O} ; \mathbf{z} \preceq \mathbf{x}, \sum_{i=1}^9 z_i + 1 = \sum_{i=1}^9 x_i \right\}, \quad
   \mathsf{Ch}(\mathbf{x}) = \left\{ \mathbf{z} \in \mathcal{O} ; \mathbf{x} \preceq \mathbf{z}, \sum_{i=1}^9 x_i + 1 = \sum_{i=1}^9 z_i \right\} $$
$$ \mathsf{Anc}(\mathbf{x}) = \left\{ \mathbf{z} \in \mathcal{O} ; \mathbf{z} \preceq \mathbf{x} \right\}, \quad
   \mathsf{Des}(\mathbf{x}) = \left\{ \mathbf{z} \in \mathcal{O} ; \mathbf{x} \preceq \mathbf{z} \right\} $$
有了这个偏序关系后，在搜索最佳宝石分配方案时，一旦宝石储量可以满足某个方案，那么我们就没有必要去对这个方案的子孙进行计算了。

### 8.2.1 基于Broad First Search的算法
由于 $ \mathbb{G}_i $ 是按照宝石组合带来的额外得分降序排序的，一旦我们发现宝石储量可以满足宝石分配方案 $ \mathsf{x} $，那么就无需考虑它的所有子孙分配方案了。
记 $\mathcal{Q}(\mathbf{x})$为宝石分配方案$\mathbf{x}$对应的额外得分，算法可以归纳为以下几步：

1. 对每张卡算出 $ \mathbb{G}_i $，得到搜素空间 $ \mathcal{O} \triangleq \bigotimes_{i=1}^9 \{ 1, 2, \ldots, | \mathbb{G}_i | \}  $
2. 构造有向图 $ \mathcal{G}=(V,E) $，其中节点集合 $ V = \mathcal{O} $，有向边集合 $ E = \{ \mathbf{x} \to \mathbf{y} ; \mathbf{y} \in \mathsf{Ch}(\mathbf{x}), \mathbf{x},\mathbf{y} \in \mathcal{O} \} $
3. 在$ \mathcal{G} $ 上使用广度优先遍历算法：
   1. 将 $[1,1,\ldots,1]$ 加入队列，初始化最强方案 $\mathbf{x}_{\mathrm{opt}} = \text{None}$ 及其额外得分 $ Q_{\mathrm{max}} = 0 $
   2. 从队列中拿出第一个宝石分配方案 $\mathbf{x}$
      * 如果宝石储量无法满足 $\mathbf{x}$， 对每个 $ \mathbf{z} \in \mathsf{Ch}(\mathbf{x}) $，如果 $\mathbf{z}$ 没被访问过，标记 $\mathbf{z}$ 为已访问，如果 $ \mathcal{Q}(\mathbf{z}) \ge Q_{\mathrm{max}} $ 则将其加入队列
      * 如果宝石储量可以满足 $\mathbf{x}$， 如果 $ \mathcal{Q}(\mathbf{x}) > Q_{\mathrm{max}} $，更新 $ \mathbf{x}_{\mathrm{opt}} \gets \mathbf{x} $，$ Q_{\mathrm{max}} \gets \mathcal{Q}(\mathbf{x}) $。
   3. 如果此时队列为空，遍历结束，反之回到步骤B

### 8.2.2 基于Divide & Conquer的算法
当强度比较高的宝石数量很少时，基于BFS的算法会将大量不可行的方案加入队列导致算法过慢。
使用Divide & Conquer的话，每次我们会找到“最稀缺”的宝石种类 $\text{gem}$，假设最稀缺宝石 $\text{gem}$ 的持有量是 $g$ ，将现有问题分割成 $ \binom{9}{g} $ 个子问题，每个只考虑将这类宝石放在特定的 $g$ 张卡中，如此递归计算。

1. 对每张卡算出 $ \mathbb{G}_i $，得到搜素空间 $ \mathcal{O} \triangleq \bigotimes_{i=1}^9 \mathrm{O}_i = \bigotimes_{i=1}^9 \{ 1, 2, \ldots, | \mathbb{G}_i | \}  $
2. 在 $ \mathcal{O} $ 上使用Divide & Conquer：

   1. 如果宝石储量满足 $ \mathcal{O} $ 中最强的方案 $\mathbf{x}$ ，且 $ \mathcal{Q}(\mathbf{x}) > Q_{\mathrm{max}} $，更新 $ \mathbf{x}_{\mathrm{opt}} \gets \mathbf{x} $，$ Q_{\mathrm{max}} \gets \mathcal{Q}(\mathbf{x}) $。
   2. 如果宝石储量不满足 $ \mathcal{O} $ 中最强的方案 $\mathbf{x}$，找出 $\mathbf{x}$ 最稀缺的宝石种类 $\text{gem}$，假设最稀缺宝石 $\text{gem}$ 的持有量是 $g$ ，将现有问题分割成 $ \binom{9}{g} $ 个子问题，每个子问题可以用一个长度为9，恰好有 $g$ 个1的二进制序列 $\mathbf{s}$ 表示，其对应的搜索空间为
   $ \mathcal{O}_{\mathbf{s}} = \bigotimes_{i=1}^9 \mathrm{O}_{i,s_i} $。
   其中 $ \mathrm{O}_{i,1} = \mathrm{O}_i $，$ \mathrm{O}_{i,0} $ 由 $ \mathrm{O}_i $ 去掉所有包含 $\text{gem}$ 的方案得到。
   3. 对每一个子问题计算其最强方案的额外得分，如果大于 $Q_{\mathrm{max}}$，则接着对这个子问题使用Divide & Conquer。

### 8.2.3 基于Dynamical Programming的算法
当所有宝石的数量都很少时，基于Divide & Conquer的算法会分割出过多的子问题（考虑9张8槽卡，每种宝石只有2个，最终子问题个数大概会有 $ \binom{9}{2}^{宝石种类个数} $）。
注意到可以独立地给每个宝石计算分配方案，LL Helper使用了基于Dynamical Programming的算法来避免计算不可行的方案。

1. 对每张卡算出 $ \mathbb{G}_i $，得到搜素空间 $ \mathcal{O} \triangleq \bigotimes_{i=1}^9 \mathrm{O}_i = \bigotimes_{i=1}^9 \{ 1, 2, \ldots, | \mathbb{G}_i | \}  $。
2. 初始宝石储量向量 $\mathbf{r}^0$，记每个单卡宝石分配对应的宝石消费向量为 $ \mathbf{c}(\text{alloc}) $，单卡宝石分配的额外得分 $ q(i,\text{alloc}) $。
3. 构造一个有9个Stage的Trellis图，节点为集合 $ \{ (i, \mathrm{r}) \,;\, i \in \{1,\ldots,9\}, \mathbf{r} \preceq \mathbf{r}^0 \} $，我们接下来需要做的是对于每一个节点 $(i, \mathbf{r})$ 记录前 $i$ 张卡分配后，剩余宝石储量向量为 $\mathbf{r}$时的 最强的分配方案 $ \text{Plan}(i,\mathbf{r}) $ 及其额外得分 $ Q(i, \mathbf{r}) $。
4.对 Stage $i$，$i \in \{1,\ldots,9\}$，作如下计算
$$ [\mathbf{r}_\star, \text{alloc}_\star] = \arg\max_{\mathbf{r}^\prime \preceq \mathbf{r}^0, \text{alloc}^\prime \in \textrm{O}_i} {\large\{} Q(i-1,\mathbf{r}^\prime) + q(i, \text{alloc}^\prime) \,;\, \mathbf{r}^\prime - \mathbf{c}(\text{alloc}^\prime) = \mathbf{r} \}  $$
$$ Q(i, \mathbf{r}) = Q(i-1,\mathbf{r}_\star) + q(i, \text{alloc}_\star), \qquad
   \text{Plan}(i, \mathbf{r}) = [\text{Plan}(i-1,\mathbf{r}_\star) \,,\, \text{alloc}_\star]
$$
5. 计算完9个Stage以后，最佳分配方案为
$$ \mathbf{r}_\star = \arg\max_{\mathbf{r}^\prime \preceq \mathbf{r}^0} Q(9, \mathbf{r}^\prime), \qquad 
   \text{Plan}_\star = \text{Plan}(9,\mathbf{r}_\star)
$$

由于Dynamical Programming的算法并没有用到之前对每张卡分配方案对排序，我们可以省掉排序的计算量。但是同时我们也没办法利用排序的结果进一步避免不必要的运算，因此在不怎么缺宝石的情况下Divide & Conquer要快一些，不过综上所述Dynamical Programming对于大多数情况是最好的算法。

## 8.3 搜索最优卡组
目前我们已经知道固定C位卡和其他8张卡后如何获得最优宝石分配，下一步的目标是寻找最最优的8张卡组成队伍并计算最优宝石分配

### 8.3.1 Brute Force解法
固定C位卡以后，在【预备卡组】中选择剩下的8张卡一共有 $\binom{K}{8}$ 种情况，我们可以对每种情况都使用 Section 8.1 的算法然后找强度最高的队伍&宝石组合方案。

### 8.3.2 快速的次优解法
在 $K$ 很大，或者 Section 8.2 算法运行速度不快的时候，Brute Force的解法会非常耗时，这里笔者介绍一种次优的但是计算速度更快的方法。

定义【$t$-邻居卡组】：如果两个卡组最多有 $t$ 个非C位的卡不同，那么这两个卡组互为【$t$-邻居卡组】。

定义【$t$-step suboptimal】：给定一个卡组，如果这个卡组的总得分强于任意一个它的【$t$-邻居卡组】，那么我们称这个卡组是 【$t$-step suboptimal】的。

需要注意的是，有可能一次性改变$t+1$张或以上的卡得到更高的总得分，所以这个算法是次优的。
例如考虑【1-step suboptimal】的情形，将一张非判卡换成判卡，对卡组提升不大，而将一张非判卡换成判卡，一定程度上提高了团队期望覆盖率从而使得装备诡计宝石的额外得分有所提高。

确定【预备卡组】之后，通过以下算法搜索【$t$-step suboptimal】的解：
1. 选择【C位卡】加上【预备卡组】里的前8张作为临时卡组，通过Section 8.2计算【临时卡组】最佳宝石以及总得分，将其放入待处理队列中
2. 将队列里的第一个卡组提出来作为【临时卡组】，通过Section 8.2计算其每个【$t$-邻居卡组】的总得分，
   * 如果【邻居卡组】强于【临时卡组】且不在队列中，将其加入队列
   * 如果【邻居卡组】强于目前最强的卡组，将最强卡组更新
5. 如果此时队列为空，算法结束，反之回到第2步

根据笔者的经验，当 $t=4$ 是基本可以保证得到的结果与Brute Force相同。